In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [ ]:
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
train_drug = pd.read_csv("/kaggle/input/lish-moa/train_drug.csv")
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_targets_nonscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

In [ ]:
print(train_drug.shape)
train_drug.head()

In [ ]:
print(train_features.shape)
train_features.head()

In [ ]:
print(train_targets_nonscored.shape)
train_targets_nonscored.head()

In [ ]:
print(train_targets_scored.shape)
train_targets_scored.head()

In [ ]:
print(test_features.shape)
test_features.head()

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
print(train_targets_scored.columns)
print("*****************************")
print(submission.columns)

In [ ]:
print(train_features['cp_type'].unique(), train_features['cp_time'].unique(), train_features['cp_dose'].unique())

In [ ]:
X = train_features.drop(['sig_id'], axis=1)
y = train_targets_scored.drop(['sig_id'], axis=1)
test = test_features.drop(['sig_id'], axis=1)

In [ ]:
print(X.isnull().sum().sum(), y.isnull().sum().sum(), test.isnull().sum().sum())

In [ ]:
lb = LabelBinarizer()
lb.fit(X['cp_type'])
print(lb.classes_)
X['cp_type'] = lb.transform(X['cp_type'])
test['cp_type'] = lb.transform(test['cp_type'])

In [ ]:
lb1 = LabelBinarizer()
lb1.fit(X['cp_dose'])
print(lb1.classes_)
X['cp_dose'] = lb1.transform(X['cp_dose'])
test['cp_dose'] = lb1.transform(test['cp_dose'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
test = sc.transform(test)

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(.9)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
test = pca.transform(test)

In [ ]:
percentage_variation = np.round(pca.explained_variance_ratio_ * 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(percentage_variation)+1)]
len(percentage_variation)

In [ ]:
plt.bar(x=range(1,len(percentage_variation)+1), height=percentage_variation, tick_label=labels)
plt.ylabel("Percentage of Explained Varience")
plt.xlabel("Principal component")
plt.title('Screen Plot')
plt.show()

In [ ]:
from datetime import datetime
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
start = datetime.now().replace(microsecond=0)
classifier_XG = xgb.XGBClassifier(objective='binary:logistic')
classifier_XG_MO = MultiOutputClassifier(classifier_XG)
classifier_XG_MO.fit(X_train, y_train)
end = datetime.now().replace(microsecond=0)
print(end-start)

In [ ]:
predictions_XG = classifier_XG_MO.predict(X_test)
print("Accuracy = ",accuracy_score(y_test,predictions_XG))

In [ ]:
predictions_test = classifier_XG_MO.predict(test)

In [ ]:
print(predictions_test.shape)
sub2 = pd.DataFrame(predictions_test)

In [ ]:
sub1 = submission['sig_id']

In [ ]:
final = pd.concat([sub1, sub2], axis = 1)
final.columns = submission.columns

In [ ]:
final.to_csv('submission.csv', index=False)